<a href="https://colab.research.google.com/github/mariajosenore/aplicacion_IA/blob/main/Student%20Puntualic%20Dataset/Student_Punctuality_Logistic_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Predicción de impuntualidad estudiantil con Regresión Logística
> **Objetivo:** predecir si un estudiante **llegará tarde** a su primera clase del día a partir de hábitos de sueño, traslado y costumbres.  
> **Modelo base:** Regresión Logística (clasificación binaria).  
> **Dataset:** `student_punctuality.csv` (sintético, 1,200 filas).

---


In [ ]:

# !pip install scikit-learn matplotlib pandas
import pandas as pd

url = "https://raw.githubusercontent.com/mariajosenore/aplicacion_IA/main/Student%20Punctuality%20Dataset/student_punctuality.csv"
data = pd.read_csv(url)
data.head()

import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                             confusion_matrix, classification_report, roc_auc_score,
                             RocCurveDisplay)


## 1. Cargar el dataset

In [ ]:

DATA_PATH = 'student_punctuality.csv'

df = pd.read_csv(DATA_PATH)
df.head()


In [ ]:

df.info()


In [ ]:

df.describe(include='all')


## 2. Exploración rápida

In [ ]:

# Balance de clases
class_counts = df['late'].value_counts().sort_index()
plt.figure()
class_counts.plot(kind='bar')
plt.title('Balance de clases (0 = puntual, 1 = tarde)')
plt.xlabel('late')
plt.ylabel('frecuencia')
plt.show()

print('Proporción de tarde:', df['late'].mean().round(3))


In [ ]:

# Correlaciones de numéricas (inspección rápida)
num_cols = df.select_dtypes(include='number').columns.tolist()
corr = df[num_cols].corr()
plt.figure()
plt.imshow(corr, interpolation='nearest')
plt.xticks(range(len(num_cols)), num_cols, rotation=90)
plt.yticks(range(len(num_cols)), num_cols)
plt.title('Matriz de correlación (numéricas)')
plt.colorbar()
plt.tight_layout()
plt.show()


## 3. Preparación de datos y pipeline

In [ ]:

X = df.drop(columns=['late'])
y = df['late']

numeric_features = X.select_dtypes(include='number').columns.tolist()
categorical_features = X.select_dtypes(exclude='number').columns.tolist()

preprocess = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), categorical_features),
    ]
)

logreg = LogisticRegression(max_iter=200, class_weight=None, n_jobs=None)

pipe = Pipeline(steps=[('prep', preprocess),
                      ('model', logreg)])

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


## 4. Entrenamiento

In [ ]:

pipe.fit(X_train, y_train)

y_pred = pipe.predict(X_test)
y_proba = pipe.predict_proba(X_test)[:, 1]

print('Accuracy :', accuracy_score(y_test, y_pred).round(3))
print('Precision:', precision_score(y_test, y_pred).round(3))
print('Recall   :', recall_score(y_test, y_pred).round(3))
print('F1       :', f1_score(y_test, y_pred).round(3))
print('ROC AUC  :', roc_auc_score(y_test, y_proba).round(3))

print('\nClassification report:\n', classification_report(y_test, y_pred))

cm = confusion_matrix(y_test, y_pred)
plt.figure()
plt.imshow(cm, cmap=None)
plt.title('Matriz de confusión')
plt.xticks([0,1], ['Pred 0','Pred 1'])
plt.yticks([0,1], ['True 0','True 1'])
for i in range(2):
    for j in range(2):
        plt.text(j, i, cm[i, j], ha='center', va='center')
plt.xlabel('Predicción')
plt.ylabel('Real')
plt.tight_layout()
plt.show()

RocCurveDisplay.from_estimator(pipe, X_test, y_test)
plt.title('Curva ROC')
plt.show()


## 5. Validación cruzada

In [ ]:

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(pipe, X, y, cv=cv, scoring='f1')
print('F1 en CV (5 folds):', scores.round(3))
print('Media F1:', scores.mean().round(3))


## 6. Interpretación de coeficientes (odds ratio)

In [ ]:

# Recuperar nombres de features después del preprocesamiento
ohe = pipe.named_steps['prep'].named_transformers_['cat']
cat_names = []
if len(ohe.categories_) > 0:
    for cats, name in zip(ohe.categories_, X[categorical_features].columns):
        # drop='first' => se omite la primera categoría
        cat_names.extend([f'{name}={c}' for c in cats[1:]])

feat_names = numeric_features + cat_names

coefs = pipe.named_steps['model'].coef_[0]
odds = np.exp(coefs)
coef_df = pd.DataFrame({'feature': feat_names, 'coef': coefs, 'odds_ratio': odds}).sort_values('odds_ratio', ascending=False)
coef_df.head(12)


In [ ]:

# Visualización simple de los top-12 (por |coef|)
top = coef_df.reindex(coef_df['coef'].abs().sort_values(ascending=False).index)[:12]
plt.figure()
plt.barh(top['feature'], top['coef'])
plt.gca().invert_yaxis()
plt.title('Top 12 coeficientes (signo indica efecto)')
plt.tight_layout()
plt.show()


## 7. Ajuste del umbral de decisión

In [ ]:

thresholds = np.linspace(0.1, 0.9, 17)
metrics = []
for t in thresholds:
    pred_t = (y_proba >= t).astype(int)
    metrics.append((t,
                    precision_score(y_test, pred_t, zero_division=0),
                    recall_score(y_test, pred_t, zero_division=0),
                    f1_score(y_test, pred_t, zero_division=0)))
m = np.array(metrics)
best_idx = m[:,3].argmax()
print('Mejor F1 con umbral:', m[best_idx,0].round(2), '→ F1 =', m[best_idx,3].round(3))
plt.figure()
plt.plot(m[:,0], m[:,1], label='Precision')
plt.plot(m[:,0], m[:,2], label='Recall')
plt.plot(m[:,0], m[:,3], label='F1')
plt.xlabel('Umbral')
plt.ylabel('Métrica')
plt.legend()
plt.title('Trade-off con el umbral de decisión')
plt.show()


## 8. Guardar el modelo

In [ ]:

import joblib
joblib.dump(pipe, 'logreg_student_punctuality.joblib')
print('Modelo guardado como logreg_student_punctuality.joblib')



## 9. Conclusiones y siguientes pasos
- La **Regresión Logística** funciona bien como línea base y permite interpretar efectos mediante coeficientes (odds ratio).
- Si necesitas capturar relaciones no lineales, intenta **árboles**, **Random Forest**, **Gradient Boosting** o **XGBoost**, y compara con validación cruzada.
- Considera aspectos de **balance de clases** (por ejemplo, `class_weight='balanced'` o *resampling*) si tu dataset real está muy desbalanceado.
- Repite el análisis con datos reales (encuestas a estudiantes) para validar el comportamiento del modelo.

> **Cita del dataset:** Este dataset es sintético y se genera con fines educativos.
